# Import Statements

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
import dominate
from dominate.tags import *
import espn_api.football
from espn_api.football import League, Player
import time
import pickle
import uuid
import drawsvg as draw
from typing import Literal

from python import constants, functions

# Testing and Debugging

In [ ]:
data = functions.summary_table(data=constants.GAME_DATA, year=2023)
x_col = 'Year'
y_col = 'Luck Score'
width = 500, 
height = 300,
x_tick_spacing = 50,
y_tick_spacing = 50

d = draw.Drawing(width=width, height=height)

border = draw.Lines(0, 0, width, 0, width, height, 0, height, close=True, fill='white', id='border')
d.append(border)

# margin x and y from edges of visual
m_top, m_bottom, m_left, m_right = 10, 50, 60, 10
# plot width and height
P_x, P_y = width - m_left - m_right, height - m_top - m_bottom
# tick margin (distance between axes and tick label)
m_tick = 5

x_min, x_max = int(np.floor(data[x_col].min() / x_tick_spacing) * x_tick_spacing), int(np.ceil(data[x_col].max() / x_tick_spacing) * x_tick_spacing)
y_min, y_max = int(np.floor(data[y_col].min() / y_tick_spacing) * y_tick_spacing), int(np.ceil(data[y_col].max() / y_tick_spacing) * y_tick_spacing)

x_ticks = [(i * x_tick_spacing) + x_min for i in range(1, int((x_max - x_min) / x_tick_spacing))]
y_ticks = [(i * y_tick_spacing) + y_min for i in range(1, int((y_max - y_min) / y_tick_spacing))]

for xtick in x_ticks:
    d.append(draw.Line(m_left + (xtick - x_min) / (x_max - x_min) * P_x, m_top, m_left + (xtick - x_min) / (x_max - x_min) * P_x, height - m_bottom, stroke='lightgrey'))
    d.append(draw.Text(str(xtick), font_size=10, x=m_left + (xtick - x_min) / (x_max - x_min) * P_x, y=height - m_bottom + m_tick, text_anchor='middle', dominant_baseline='hanging', font_family='Arial'))
for ytick in y_ticks:
    d.append(draw.Line(m_left, (height - m_bottom) - (ytick - y_min) / (y_max - y_min) * P_y, width - m_right, (height - m_bottom) - (ytick - y_min) / (y_max - y_min) * P_y, stroke='lightgrey'))
    d.append(draw.Text(str(ytick), font_size=10, x=m_left - m_tick, y=(height - m_bottom) - (ytick - y_min) / (y_max - y_min) * P_y, text_anchor='end', dominant_baseline='middle', font_family='Arial'))

axes = draw.Lines(m_left, m_top, width - m_right, m_top, width - m_right, height - m_bottom, m_left, height - m_bottom, close=True, fill='none', stroke='black', id='axes')
d.append(axes)

x_label = draw.Text(x_col, font_size=16, x=(P_x / 2 + m_left), y=(height - (m_bottom / 2)), text_anchor='middle', dominant_baseline='hanging', font_family='Arial')
d.append(x_label)
y_label = draw.Text(y_col, font_size=16, x=10, y=(P_y / 2 + m_top), text_anchor='middle', dominant_baseline='hanging', transform=f'rotate(-90, {10}, {P_y / 2 + m_top})', font_family='Arial')
d.append(y_label)

for index, row in data.iterrows():
    v_x = m_left + ((row[x_col] - x_min) / (x_max - x_min) * (P_x))
    v_y = (height - m_bottom) - ((row[y_col] - y_min) / (y_max - y_min) * (P_y))

    d.append(draw.Circle(v_x, v_y, r=4, fill=constants.COLOR_DICT[row['Team'].lower()], stroke='black', stroke_width=1.5))

result = d.as_svg()



# with open('test.svg', 'w') as file:
#     file.write(d.as_svg())

d.display_image()


# print(x_min, x_max, y_min, y_max)
# print(x_ticks)
# print(y_ticks)

/var/folders/b5/lkkh5rx9771c1mbt1ckv105w0000gn/T/ipykernel_25934/2675656509.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_min, x_max = int(np.floor(data[x_col].min() / x_tick_spacing) * x_tick_spacing), int(np.ceil(data[x_col].max() / x_tick_spacing) * x_tick_spacing)


TypeError: unsupported operand type(s) for /: 'int' and 'tuple'

In [ ]:
def round_min(value, rounding, min_distance=0.1):
    rounded = np.floor(value / rounding) * rounding

    if value - rounded < rounding * min_distance:
        rounded -= rounding

    return int(rounded)

def round_max(value, rounding, min_distance=0.1):
    rounded = np.ceil(value / rounding) * rounding

    if rounded - value < rounding * min_distance:
        rounded += rounding

    return int(rounded)

def df_to_svg(
        data: pd.DataFrame, 
        x_col: str, 
        y_col: str,
        chart_type: str = 'scatter',
        width: int = 500, 
        height: int = 300,
        x_tick_spacing: int = 50,
        y_tick_spacing: int = 50
):

    d = draw.Drawing(width=width, height=height)

    border = draw.Lines(0, 0, width, 0, width, height, 0, height, close=True, fill='white', id='border')
    d.append(border)

    # margin x and y from edges of visual
    m_top, m_bottom, m_left, m_right = 10, 50, 60, 10
    # plot width and height
    P_x, P_y = width - m_left - m_right, height - m_top - m_bottom
    # tick margin (distance between axes and tick label)
    m_tick = 5

    x_min, x_max = round_min(data[x_col].min(), rounding=x_tick_spacing), round_max(data[x_col].max(), rounding=x_tick_spacing)
    y_min, y_max = round_min(data[y_col].min(), rounding=y_tick_spacing), round_max(data[y_col].max(), rounding=y_tick_spacing)

    x_ticks = [(i * x_tick_spacing) + x_min for i in range(1, int((x_max - x_min) / x_tick_spacing))]
    y_ticks = [(i * y_tick_spacing) + y_min for i in range(1, int((y_max - y_min) / y_tick_spacing))]

    grid_color = 'lightgrey'
    for xtick in x_ticks:
        d.append(draw.Line(m_left + (xtick - x_min) / (x_max - x_min) * P_x, m_top, m_left + (xtick - x_min) / (x_max - x_min) * P_x, height - m_bottom, stroke=grid_color))
        d.append(draw.Text(str(xtick), font_size=10, x=m_left + (xtick - x_min) / (x_max - x_min) * P_x, y=height - m_bottom + m_tick, text_anchor='middle', dominant_baseline='hanging', font_family='Arial'))
    for ytick in y_ticks:
        if ytick == 0:
            d.append(draw.Line(m_left, (height - m_bottom) - (ytick - y_min) / (y_max - y_min) * P_y, width - m_right, (height - m_bottom) - (ytick - y_min) / (y_max - y_min) * P_y, stroke='black'))
        else:
            d.append(draw.Line(m_left, (height - m_bottom) - (ytick - y_min) / (y_max - y_min) * P_y, width - m_right, (height - m_bottom) - (ytick - y_min) / (y_max - y_min) * P_y, stroke=grid_color))
        d.append(draw.Text(str(ytick), font_size=10, x=m_left - m_tick, y=(height - m_bottom) - (ytick - y_min) / (y_max - y_min) * P_y, text_anchor='end', dominant_baseline='middle', font_family='Arial'))

    axes = draw.Lines(m_left, m_top, width - m_right, m_top, width - m_right, height - m_bottom, m_left, height - m_bottom, close=True, fill='none', stroke='black', id='axes')
    d.append(axes)

    x_label = draw.Text(x_col, font_size=16, x=(P_x / 2 + m_left), y=(height - (m_bottom / 2)), text_anchor='middle', dominant_baseline='hanging', font_family='Arial')
    d.append(x_label)
    y_label = draw.Text(y_col, font_size=16, x=10, y=(P_y / 2 + m_top), text_anchor='middle', dominant_baseline='hanging', transform=f'rotate(-90, {10}, {P_y / 2 + m_top})', font_family='Arial')
    d.append(y_label)

    points = []
    circles = []
    for index, row in data.iterrows():
        v_x = m_left + ((row[x_col] - x_min) / (x_max - x_min) * (P_x))
        v_y = (height - m_bottom) - ((row[y_col] - y_min) / (y_max - y_min) * (P_y))

        circles.append(draw.Circle(v_x, v_y, r=4, fill=constants.COLOR_DICT[row['Team'].lower()], stroke='black', stroke_width=1.5))
        
        if chart_type == 'line':
            points.extend([v_x, v_y])
    
    if chart_type == 'line':
        d.append(draw.Lines(*points, fill='none', stroke='black'))
    
    for circle in circles:
        d.append(circle)

    result = d.as_svg()

    # raw_svg = d.as_svg().split('\n')
    # raw_svg[1] = ' '.join([p for p in raw_svg[1].split(' ') if not p.startswith('xmlns:xlink')])
    # raw_svg[1] = raw_svg[1].strip() + ' ' + raw_svg[2].strip()
    # del raw_svg[2]

    # svg_str = []

    # for line in raw_svg:
    #     if line.startswith('<?xml') or line == '<defs>' or line == '</defs>':
    #         continue
    #     if line.startswith('<svg') or line == '</svg>':
    #         svg_str.append(line)
    #         continue
    #     svg_str.append('  ' + line)

    # result = dominate.util.raw('\n' + '\n'.join(svg_str))

    return result

seasons = [functions.summary_table(constants.GAME_DATA, year=year) for year in constants.YEARS]
seasons_df = pd.concat(seasons)
seasons_df = seasons_df.loc[seasons_df['Team'] == 'Ethan']

clean_svg = df_to_svg(
    data=seasons_df,
    x_col='Year',
    y_col='Luck Score',
    chart_type='line',
    x_tick_spacing=1,
    y_tick_spacing=2)

# with open('test.svg', 'w') as file:
#     file.write(clean_svg)

print(clean_svg)
clean_svg.display_image()

JupyterSvgImage(svg='<?xml version="1.0" encoding="UTF-8"?>\n<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"\n     width="500" height="300" viewBox="0 0 500 300">\n<defs>\n</defs>\n<path d="M0,0 L500,0 L500,300 L0,300 Z" fill="white" id="border" />\n<path d="M121.42857142857142,10 L121.42857142857142,250" stroke="lightgrey" />\n<text x="121.42857142857142" y="255" font-size="10" text-anchor="middle" dominant-baseline="hanging" font-family="Arial">2019</text>\n<path d="M182.85714285714283,10 L182.85714285714283,250" stroke="lightgrey" />\n<text x="182.85714285714283" y="255" font-size="10" text-anchor="middle" dominant-baseline="hanging" font-family="Arial">2020</text>\n<path d="M244.28571428571428,10 L244.28571428571428,250" stroke="lightgrey" />\n<text x="244.28571428571428" y="255" font-size="10" text-anchor="middle" dominant-baseline="hanging" font-family="Arial">2021</text>\n<path d="M305.71428571428567,10 L305.71428571428567,250" stroke="lightgrey" />\n<text x="305.71428571428567" y="255" font-size="10" text-anchor="middle" dominant-baseline="hanging" font-family="Arial">2022</text>\n<path d="M367.14285714285717,10 L367.14285714285717,250" stroke="lightgrey" />\n<text x="367.14285714285717" y="255" font-size="10" text-anchor="middle" dominant-baseline="hanging" font-family="Arial">2023</text>\n<path d="M428.57142857142856,10 L428.57142857142856,250" stroke="lightgrey" />\n<text x="428.57142857142856" y="255" font-size="10" text-anchor="middle" dominant-baseline="hanging" font-family="Arial">2024</text>\n<path d="M60,236.66666666666666 L490,236.66666666666666" stroke="lightgrey" />\n<text x="55" y="236.66666666666666" font-size="10" text-anchor="end" dominant-baseline="middle" font-family="Arial">-14</text>\n<path d="M60,223.33333333333334 L490,223.33333333333334" stroke="lightgrey" />\n<text x="55" y="223.33333333333334" font-size="10" text-anchor="end" dominant-baseline="middle" font-family="Arial">-12</text>\n<path d="M60,210.0 L490,210.0" stroke="lightgrey" />\n<text x="55" y="210.0" font-size="10" text-anchor="end" dominant-baseline="middle" font-family="Arial">-10</text>\n<path d="M60,196.66666666666669 L490,196.66666666666669" stroke="lightgrey" />\n<text x="55" y="196.66666666666669" font-size="10" text-anchor="end" dominant-baseline="middle" font-family="Arial">-8</text>\n<path d="M60,183.33333333333331 L490,183.33333333333331" stroke="lightgrey" />\n<text x="55" y="183.33333333333331" font-size="10" text-anchor="end" dominant-baseline="middle" font-family="Arial">-6</text>\n<path d="M60,170.0 L490,170.0" stroke="lightgrey" />\n<text x="55" y="170.0" font-size="10" text-anchor="end" dominant-baseline="middle" font-family="Arial">-4</text>\n<path d="M60,156.66666666666669 L490,156.66666666666669" stroke="lightgrey" />\n<text x="55" y="156.66666666666669" font-size="10" text-anchor="end" dominant-baseline="middle" font-family="Arial">-2</text>\n<path d="M60,143.33333333333334 L490,143.33333333333334" stroke="black" />\n<text x="55" y="143.33333333333334" font-size="10" text-anchor="end" dominant-baseline="middle" font-family="Arial">0</text>\n<path d="M60,130.0 L490,130.0" stroke="lightgrey" />\n<text x="55" y="130.0" font-size="10" text-anchor="end" dominant-baseline="middle" font-family="Arial">2</text>\n<path d="M60,116.66666666666666 L490,116.66666666666666" stroke="lightgrey" />\n<text x="55" y="116.66666666666666" font-size="10" text-anchor="end" dominant-baseline="middle" font-family="Arial">4</text>\n<path d="M60,103.33333333333331 L490,103.33333333333331" stroke="lightgrey" />\n<text x="55" y="103.33333333333331" font-size="10" text-anchor="end" dominant-baseline="middle" font-family="Arial">6</text>\n<path d="M60,90.0 L490,90.0" stroke="lightgrey" />\n<text x="55" y="90.0" font-size="10" text-anchor="end" dominant-baseline="middle" font-family="Arial">8</text>\n<path d="M60,76.66666666666666 L490,76.66666666666666" stroke="lightgrey" />\n<text x="55" y="76.66666666666666"

In [19]:
def summary_table(data: pd.DataFrame, year: int, week: int = None) -> pd.DataFrame:
    temp = data.loc[(data['Year'] == year) & (data['Playoff Flag'] == False)].copy()

    if week != None:
        temp = temp.loc[(data['Week'] <= week)]

    mean = temp['Score'].mean()
    std = temp['Score'].std() * 0.5

    opp_luck = []
    your_luck = []
    close_luck = []

    for your_score, opp_score in temp[['Score','Opp Score']].values:
        opp_luck.append(functions.opp_luck_score(opp_score=opp_score, mean=mean, std=std))
        your_luck.append(functions.your_luck_score(your_score=your_score, opp_score=opp_score, mean=mean, std=std))
        close_luck.append(functions.close_luck_score(your_score=your_score, opp_score=opp_score))

    temp['Opp Luck Score'] = opp_luck
    temp['Your Luck Score'] = your_luck
    temp['Close Luck Score'] = close_luck

    temp['Luck Score'] = temp[['Opp Luck Score','Your Luck Score','Close Luck Score']].sum(axis=1)

    champ_week = data.loc[data['Year'] == year, 'Week'].max()
    champ = data.loc[(data['Year'] == year) & (data['Week'] == champ_week) & (data['Win'] == 1), 'Team'].item()

    league_pfpg = round(temp['Score'].mean(), 2)

    temp_teams = temp['Team'].unique()
    weekly_standings = []

    for team in temp_teams:
        temp_team = temp.loc[temp['Team'] == team]
        wins = temp_team['Win'].sum()
        losses = temp_team['Win'].eq(0).sum()
        record = f'{wins}-{losses}'

        pf = round(temp_team['Score'].sum(), 2)
        pfpg = round(temp_team['Score'].mean(), 2)
        pfpg_plus = int(pfpg / league_pfpg * 100)
        pa = round(temp_team['Opp Score'].sum(), 2)
        papg = round(temp_team['Opp Score'].mean(), 2)
        papg_plus = int(papg / league_pfpg * 100)
        avg_margin = round((pf - pa) / len(temp_team), 2)
        luck_score = temp_team['Luck Score'].sum()

        champ_flag = int(champ == team)

        weekly_standings.append(
            {
                'Team':team,
                'Wins':wins,
                'Losses':losses,
                'Record':record,
                'Points For':pf,
                'Points Against':pa,
                'PF/G':pfpg,
                'PF/G+':pfpg_plus,
                'PA/G':papg,
                'PA/G+':papg_plus,
                'Avg Margin':avg_margin,
                'Luck Score':luck_score,
                'Champ Flag':champ_flag
            }
        )

    weekly_standings = pd.DataFrame(weekly_standings)
    weekly_standings.sort_values(['Wins','Points For'], ascending=False, ignore_index=True, inplace=True)
    weekly_standings['Ranking'] = [i + 1 for i in weekly_standings.index]
    weekly_standings['Year'] = year

    return weekly_standings

In [20]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Haris']

data

,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking,Year
8,Haris,5,8,5-8,1254.30,1304.58,96.48,97,100.35,101,-3.87,-5,0,9,2018
4,Haris,7,6,7-6,1548.30,1483.26,119.10,101,114.10,96,5.00,1,0,5,2019
9,Haris,3,10,3-10,1440.82,1773.88,110.83,91,136.45,112,-25.62,-13,0,10,2020
2,Haris,8,6,8-6,1855.22,1655.12,132.52,107,118.22,96,14.29,4,0,3,2021
6,Haris,7,7,7-7,1647.48,1579.54,117.68,103,112.82,99,4.85,-3,0,7,2022
0,Haris,12,2,12-2,1801.34,1456.30,128.67,110,104.02,89,24.65,10,1,1,2023
9,Haris,3,11,3-11,1468.60,1616.88,104.90,89,115.49,98,-10.59,1,0,10,2024


In [ ]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Haris']

averages = {
    'Year':'Total',
    'Ranking':round(data['Ranking'].mean(), 2),
    'Points For':round(data['Points For'].mean(), 2),
    'Points Against':round(data['Points Against'].mean(), 2),
    'PF/G':round(data['PF/G'].mean(), 2),
    'PF/G+':round(data['PF/G+'].mean(), 2),
    'Avg Margin':round(data['Avg Margin'].mean(), 2),
    'Luck Score':round(data['Luck Score'].mean(), 2)
}

,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking,Year
8,Haris,5,8,5-8,1254.30,1304.58,96.48,97,100.35,101,-3.87,-5,0,9,2018
4,Haris,7,6,7-6,1548.30,1483.26,119.10,101,114.10,96,5.00,1,0,5,2019
9,Haris,3,10,3-10,1440.82,1773.88,110.83,91,136.45,112,-25.62,-13,0,10,2020
2,Haris,8,6,8-6,1855.22,1655.12,132.52,107,118.22,96,14.29,4,0,3,2021
6,Haris,7,7,7-7,1647.48,1579.54,117.68,103,112.82,99,4.85,-3,0,7,2022
0,Haris,12,2,12-2,1801.34,1456.30,128.67,110,104.02,89,24.65,10,1,1,2023
9,Haris,3,11,3-11,1468.60,1616.88,104.90,89,115.49,98,-10.59,1,0,10,2024


In [24]:
def df_to_table(
        data: pd.DataFrame,
        custom_columns: list[str] = None,
        row_id_columns: list[str] = None,
        table_id: str = None,
        champ_class: bool = False,
        footer_data: pd.DataFrame = None
) -> table:
    '''
    Converts a pandas DataFrame into an HTML table with optional styling.

    If dataframe is longer than 20 rows, table will be wrapped in a "scroll-table" div.

    Parameters
    ----------
    data : pd.DataFrame
        Dataframe to be converted.

    custom_columns : list[str], default None
        Columns to be displayed in the HTML table. If set to None, all columns in DataFrame will be used.

    row_id_columns : list[str], default None
        ID to be assigned to each tr tag in the tbody.
        Accepts a list of columns, will concatenate them starting with 'row' and separating with '-'.
        If set to None, no IDs will be assigned.

    table_id : str, default None
        ID to be assigned to the main table tag. If set to None, no ID will be given.

    champ_class : bool, default False

    table_footer : str {'totals','averages','both'}

    Returns
    -------
    dominate.table or dominate.div
        Return a dominate HTML object. Can be either a table or div
    '''
    t = table()
    head = thead()
    body = tbody()
    foot = tfoot()

    records = data.to_dict('records')

    columns = data.columns
    if custom_columns:
        columns = custom_columns

    column_row = tr()
    for column in columns:
        column_row.add(th(column, _class=str(column).lower().replace(' ','-')))
    head.add(column_row)
    
    for record in records:
        data_row = tr(__pretty=False)
        if champ_class:
            if record['Champ Flag'] == 1:
                data_row['class'] = 'champ'
        if row_id_columns:
            row_id = 'row'
            for column in row_id_columns:
                row_id += ('-' + str(record[column]).lower().replace(' ',''))
            data_row['id'] = row_id

        for column in columns:
            data_row.add(td(record[column], _class=str(column).lower().replace(' ','-')))
        body.add(data_row)

    if footer_data:
        footer_records = footer_data.to_dict('records')

        for record in footer_records:
            footer_row = tr(__pretty=False)
            for column in columns:
                footer_row.add(th(record[column], _class=str(column).lower().replace(' ','-')))
            foot.add(footer_row)

    t.add(head)
    t.add(body)
    if footer_data:
        t.add(foot)
    if table_id:
        t['id'] = table_id

    if len(data) > 20:
        scroll_div = div(_class='scroll-table')
        scroll_div.add(t)
        return scroll_div

    return t

In [31]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Haris']

averages = [{
    'Year':'Total',
    'Wins':round(data['Wins'].mean(), 2),
    'Losses':round(data['Losses'].mean(), 2),
    'Ranking':round(data['Ranking'].mean(), 2),
    'Points For':round(data['Points For'].mean(), 2),
    'Points Against':round(data['Points Against'].mean(), 2),
    'PF/G':round(data['PF/G'].mean(), 2),
    'PF/G+':round(data['PF/G+'].mean(), 2),
    'Avg Margin':round(data['Avg Margin'].mean(), 2),
    'Luck Score':round(data['Luck Score'].mean(), 2)
}]

pd.DataFrame(averages)

,Year,Wins,Losses,Ranking,Points For,Points Against,PF/G,PF/G+,Avg Margin,Luck Score
0,Total,6.43,7.14,6.43,1573.72,1552.79,115.74,99.71,1.24,-0.71
